# Storytelling

In [1]:
import pandas as pd
import numpy as np



/var/folders/w_/kqymfxcs5d561j515w6_cfqh0000gn/T/ipykernel_33355/2504811429.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Creazione del Task (problema con pazienti da dati storici)

In [2]:
from surgeryschedulingunderuncertainty.task import Task
from surgeryschedulingunderuncertainty.patients_provider import PatientsFromHistoricalDataProvider
from surgeryschedulingunderuncertainty.master import Master

from surgeryschedulingunderuncertainty.implementor import StandardImplementor
from surgeryschedulingunderuncertainty.optimizer import ImplementorAdversary

In [3]:
# help(Task)

In [4]:
# Instanziare un oggetto task

task = Task(name = "My first problem",
            num_of_weeks= 2,
            num_of_patients= 300,
            robustness_risk= 0.2,
            robustness_overtime= 60, # TODO check robustness_overtime unit of measure
            urgency_to_max_waiting_days= {0: 7, 1:30, 2:60, 3:180, 4:360}, 
)

In [5]:
# Importare i dati e istanziare un oggetto patient provider

historical_data_df = pd.read_csv("../data/historical_data.csv")

patient_provider = PatientsFromHistoricalDataProvider(
                 historical_data= historical_data_df
                 )

In [6]:
historical_data_df.urgency

0       3
1       1
2       0
3       3
4       0
       ..
9155    1
9156    4
9157    1
9158    1
9159    4
Name: urgency, Length: 9160, dtype: int64

In [7]:
# Campionatura di singoli pazienti
patient = patient_provider.provide_patient()
print(patient)

# Campionatura di singoli pazienti
patient = patient_provider.provide_patient(requested_equipe='C', requested_urgency=3)
print(patient)

# Campionatura di singoli pazienti
patient = patient_provider.provide_patient(requested_equipe='B', requested_urgency=3)
print(patient)

Patient id: 7183 
 equipe: L 
 urgency: 4 
 days waiting: 273
Patient id: 1502 
 equipe: C 
 urgency: 3 
 days waiting: 157
Patient id: 1129 
 equipe: B 
 urgency: 3 
 days waiting: 95


In [8]:
# Inserire un insieme di pazienti in task
task.patients = patient_provider.provide_patient_set(quantity=300)

## Previsione del tempo operatorio

In [9]:
from surgeryschedulingunderuncertainty.predictive_model import NGBLogNormal, NGBNormal


In [10]:
patients, training = patient_provider.provide_sets(quantity=10, quantity_training=100)


In [11]:
patients


In [12]:
patients[0].features

array([2, 0, 0, ..., 0.0, 0.0, 0.0], dtype=object)

In [13]:
# instantiating the model, and training

model = NGBNormal(description="test", patients=training)

In [14]:
patients_prediction = model.predict(patients)

In [15]:
patients_prediction[0].uncertainty_profile.param_loc

78.67473379822106

In [16]:
patients_prediction[0].uncertainty_profile.param_scale

7.539118909785348

In [17]:
patients_prediction[0].uncertainty_profile.sample(10)

array([76.99766753, 88.77739105, 85.24842154, 67.88130697, 73.64934741,
       75.41898443, 94.78225632, 76.82876761, 75.34225925, 74.71887082])

## Aggiunta dei pazienti con previsione (nuovo task)

In [18]:
num_of_patients = 50

In [19]:
patients, training = patient_provider.provide_sets(quantity=num_of_patients, quantity_training=1000)


In [20]:
model = NGBLogNormal(description="test", patients=training)

patients = model.predict(patients)

In [21]:

task = Task(name = "My first problem",
            num_of_weeks= 2,
            num_of_patients= num_of_patients,
            robustness_risk= 0.2,
            robustness_overtime= 10,
            urgency_to_max_waiting_days= {0: 7, 1:30, 2:60, 3:180, 4:360}, 
            )

task.patients = patients


In [22]:
patients = task.patients
patients[1].max_waiting_days

7

In [23]:
patients[1].days_waiting

7

In [24]:
print(patients[0])

Patient id: 3128 
 equipe: H 
 urgency: 4 
 nominal duration: 104 
 days waiting: 164


## Aggiunta del master scheduling

In [25]:
or_master_schedule = pd.read_csv("../data/master_schedule_input.csv", sep = ';')
or_master_schedule

,weekday,equipes,room,duration
0,1,"C, D",or1,300
1,1,"D, C",or1,300
2,1,I,or2,600
3,2,"H, F",or1,300
4,2,"D, C",or1,300
5,2,L,or2,600
6,3,"C, D",or1,300
7,3,"F, H",or1,300
8,3,I,or1,300
9,3,"H, F",or2,600


In [26]:
master_schedule  = Master(name = "my master", table=or_master_schedule)

In [27]:
print(master_schedule)

Master 'my master' - 16 blocks on 5 days.


In [28]:
task.master_schedule = master_schedule

In [29]:
for block in master_schedule.blocks:
    print(block)
    print('----\n')
    
    

Master Block number 0
                
Equipe: ['C', 'D']
                
Duration: 300
                
Room: or1
                
Weekday: 1
                
Number in day: 1
                
----

Master Block number 1
                
Equipe: ['D', 'C']
                
Duration: 300
                
Room: or1
                
Weekday: 1
                
Number in day: 2
                
----

Master Block number 2
                
Equipe: ['I']
                
Duration: 600
                
Room: or2
                
Weekday: 1
                
Number in day: 3
                
----

Master Block number 3
                
Equipe: ['H', 'F']
                
Duration: 300
                
Room: or1
                
Weekday: 2
                
Number in day: 1
                
----

Master Block number 4
                
Equipe: ['D', 'C']
                
Duration: 300
                
Room: or1
                
Weekday: 2
                
Number in day: 2
                
----



## Soluzione del modello - Implementor Adversary

In [30]:
implementor_adversary = ImplementorAdversary(task = task, implementor=StandardImplementor(), adversary=None, description = 'Test')



In [31]:
implementor_adversary._task



In [32]:
implementor_adversary.create_instance()



In [33]:
solution = implementor_adversary.run(10)

implementor
adversary


In [34]:
implementor_adversary._instance.get(None).get('n_pats')

{None: 50}

In [35]:
solution = implementor_adversary.run_implementor()


In [36]:
for pat in (solution._blocks[0].patients):
    print(pat)

Patient id: 4401 
 equipe: D 
 urgency: 1 
 nominal duration: 71 
 days waiting: 13
Patient id: 9116 
 equipe: C 
 urgency: 3 
 nominal duration: 86 
 days waiting: 149
Patient id: 4431 
 equipe: D 
 urgency: 1 
 nominal duration: 45 
 days waiting: 28


solution._blocks[4].patients[0].uncertainty_profile.param_s

In [37]:
task.master_schedule.blocks

In [38]:
task.patients

In [39]:
task.patients[0].uncertainty_profile

## Creazione dell'ottimizzatore

In [40]:
from surgeryschedulingunderuncertainty.optimizer import VanillaImplementor
from surgeryschedulingunderuncertainty.implementor import ChanceConstraintsImplementor


In [41]:
chance_constraints_model = VanillaImplementor(task = task, 
                                              implementor=ChanceConstraintsImplementor(task = task), 
                                              description = 'Test')



In [42]:
chance_constraints_model = VanillaImplementor(task = task, 
                                              implementor=ChanceConstraintsImplementor(task = task), 
                                              description = 'Test')

In [43]:
from surgeryschedulingunderuncertainty.optimizer import VanillaImplementor, BudgetSet
from surgeryschedulingunderuncertainty.implementor import ChanceConstraintsImplementor, BSImplementor


In [44]:
bs_model = BudgetSet(task = task, 
                              implementor=BSImplementor(task = task), 
                              description = 'Test')






In [45]:
bs_model.run()

hello!
{'mean': 58.531308877183086, 'std': 3.4554520278717673, 'gamma': 9, 'time_increment': 333.3333333333333}
{'mean': 58.531308877183086, 'std': 3.4554520278717673, 'gamma': 9, 'time_increment': 333.3333333333333}
{'mean': 199.93388195843357, 'std': 11.801895638591114, 'gamma': 9, 'time_increment': 666.6666666666666}
{'mean': 61.841420254529766, 'std': 0.98692242104967, 'gamma': 9, 'time_increment': 333.3333333333333}
{'mean': 58.531308877183086, 'std': 3.4554520278717673, 'gamma': 9, 'time_increment': 333.3333333333333}
{'mean': 217.4623516321134, 'std': 7.473759351119532, 'gamma': 9, 'time_increment': 666.6666666666666}
{'mean': 58.531308877183086, 'std': 3.4554520278717673, 'gamma': 9, 'time_increment': 333.3333333333333}
{'mean': 61.841420254529766, 'std': 0.98692242104967, 'gamma': 9, 'time_increment': 333.3333333333333}
{'mean': 199.93388195843357, 'std': 11.801895638591114, 'gamma': 9, 'time_increment': 333.3333333333333}
{'mean': 61.841420254529766, 'std': 0.98692242104967, 

In [46]:
bs_model.create_instance()

hello!
{'mean': 58.531308877183086, 'std': 3.4554520278717673, 'gamma': 9, 'time_increment': 333.3333333333333}
{'mean': 58.531308877183086, 'std': 3.4554520278717673, 'gamma': 9, 'time_increment': 333.3333333333333}
{'mean': 199.93388195843357, 'std': 11.801895638591114, 'gamma': 9, 'time_increment': 666.6666666666666}
{'mean': 61.841420254529766, 'std': 0.98692242104967, 'gamma': 9, 'time_increment': 333.3333333333333}
{'mean': 58.531308877183086, 'std': 3.4554520278717673, 'gamma': 9, 'time_increment': 333.3333333333333}
{'mean': 217.4623516321134, 'std': 7.473759351119532, 'gamma': 9, 'time_increment': 666.6666666666666}
{'mean': 58.531308877183086, 'std': 3.4554520278717673, 'gamma': 9, 'time_increment': 333.3333333333333}
{'mean': 61.841420254529766, 'std': 0.98692242104967, 'gamma': 9, 'time_increment': 333.3333333333333}
{'mean': 199.93388195843357, 'std': 11.801895638591114, 'gamma': 9, 'time_increment': 333.3333333333333}
{'mean': 61.841420254529766, 'std': 0.98692242104967, 

In [47]:
bs_model._instance[None].keys()

dict_keys(['g', 'gamma', 'time_increment', 'a', 't', 'u', 'w', 'l', 'n_days', 'n_blocks', 'n_rooms', 'n_pats', 'Gamma', 'c_exclusion', 'c_delay', 'n_realizations'])

In [48]:
solution = bs_model.run_implementor()

In [49]:
for pat in (solution._blocks[2].patients):
    print(pat.uncertainty_profile.nominal_value)
    print(pat.uncertainty_profile.param_s)

269.8074043500128
0.28911782425652044
212.59926303720903
0.34447445621504097


In [50]:
chance_constraints_model._task



In [51]:
chance_constraints_model.create_instance()



In [52]:
chance_constraints_model._instance.get(None).get('n_pats')

{None: 50}

In [53]:
solution = chance_constraints_model.run_implementor()


In [54]:
chance_constraints_model._instance.get(None).get('f')

{1: 104.79345424348244,
 2: 43.4504948310814,
 3: 269.80766509262725,
 4: 50.836240900424976,
 5: 45.45570203195016,
 6: 52.3301068843628,
 7: 61.89321006422617,
 8: 44.68895614106572,
 9: 86.92001595624004,
 10: 49.7917885362025,
 11: 191.58419464781596,
 12: 71.41814326150518,
 13: 44.32216453301749,
 14: 162.44995932204787,
 15: 91.22385146459992,
 16: 42.65425234106965,
 17: 86.34609750363069,
 18: 86.34637421607354,
 19: 44.41679059261145,
 20: 48.11419652673679,
 21: 145.47527514935527,
 22: 88.61363977396317,
 23: 63.527900317013376,
 24: 92.27218498878946,
 25: 42.83888129443252,
 26: 92.27239441980183,
 27: 196.3295734366252,
 28: 104.07258025397705,
 29: 97.36351131425297,
 30: 48.48480057846943,
 31: 45.12404402250023,
 32: 140.8915072790951,
 33: 50.866687858082464,
 34: 173.9155477439028,
 35: 102.9400124455927,
 36: 191.58419464781596,
 37: 111.64048737822093,
 38: 198.71161719491909,
 39: 86.34638174143372,
 40: 344.91678777101885,
 41: 207.7883932647994,
 42: 48.0715785

In [55]:
for pat in (solution._blocks[0].patients):
    print(pat)

Patient id: 4401 
 equipe: D 
 urgency: 1 
 nominal duration: 71 
 days waiting: 13
Patient id: 9116 
 equipe: C 
 urgency: 3 
 nominal duration: 86 
 days waiting: 149
Patient id: 4431 
 equipe: D 
 urgency: 1 
 nominal duration: 45 
 days waiting: 28


In [56]:
solution._blocks[0].patients[0].uncertainty_profile

In [57]:
solution.x[1,1]()

AttributeError: 'Schedule' object has no attribute 'x'

In [ ]:
task.master_schedule.blocks

In [ ]:
task.patients

In [ ]:
task.patients[0].uncertainty_profile